In [86]:
from datasets import load_dataset, DatasetDict
from datasets import concatenate_datasets
from IPython.display import HTML

from tqdm import tqdm
import re 
import numpy as np
from markdownify import markdownify as md

In [80]:
ds = load_dataset("HuggingFaceH4/stack-exchange-preferences", split="train", num_proc=16)
ds

Found cached dataset parquet (/home/leandro/.cache/huggingface/datasets/HuggingFaceH4___parquet/HuggingFaceH4--stack-exchange-preferences-1d2bff9ecb5ffe2a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['qid', 'question', 'answers', 'date', 'metadata'],
    num_rows: 10807695
})

In [81]:
HTML(ds[0]["question"])

In [82]:
def lang_callback(el):
    lang = el['class'][0] if el.has_attr('class') else None
    
    if not lang is None:
        lang = lang.split("-")[-1]
    return lang

In [83]:
def html2md(text):
    text = md(text, code_language_callback=lang_callback)
    text = re.sub(r"\n\s*\n", "\n\n", text).strip()
    return text.encode('utf-8', 'replace').decode()

In [84]:
for i in range(1):
    text = html2md(ds[i]["question"])
    print(text)
    print("=="*10)

I have been wanting to learn about 3D printing a long time so I really want this site to succeed but I have no previous experience with the subject. 

I was wondering how can I help the site at this early stage. I thought about asking about how to get started with 3D printing but SE explicitly discourages "easy" questions in the private beta.

What can newbies like me do for the site at this stage besides voting questions and answers?


In [85]:
np.mean([len(ds[i]["answers"])*(len(ds[i]["answers"])-1)/2 for i in range(10000)])

49.935

In [87]:
ds = ds.shuffle(seed=42)
index = list(range(len(ds)))

ds_splits = DatasetDict({
    "finetune": ds.select(index[:3_000_000]),
    "reward": ds.select(index[3_000_000:6_000_000]),
    "rl": ds.select(index[6_000_000:9_000_000]),
    "evaluation": ds.select(index[9_000_000:]),
})

In [88]:
ds_splits

DatasetDict({
    finetune: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata'],
        num_rows: 3000000
    })
    reward: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata'],
        num_rows: 3000000
    })
    rl: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata'],
        num_rows: 3000000
    })
    evaluation: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata'],
        num_rows: 1807695
    })
})

In [89]:
def binary_comparison(answers):
    """Returns tuples of answers, first always best"""
    pairs = []
    
    for i in range(len(answers)-1):
        for j in range(i+1, len(answers)):
            if answers[i]["pm_score"]>answers[j]["pm_score"]:
                pairs.append((answers[i]["text"], answers[j]["text"]))
            elif answers[i]["pm_score"]<answers[j]["pm_score"]:
                pairs.append((answers[j]["text"], answers[i]["text"]))
    return pairs

In [90]:
def preprocess(examples):
    """Cleans HTML and returns paired answers (j is better than k). Note that this returns more examples (one for each pair per question)."""
    
    MAX_PAIRS_PER_QUESTION = 10
    n_samples = len(examples["qid"])
    
    # initialize empty lists for new samples
    new_examples = {"question": [], "response_j": [], "response_k": []}
    for key in examples:
        new_examples[key] = []
    
    for sample_id in range(n_samples):
        # get pairs where first is always the better one
        pairs = binary_comparison(examples["answers"][sample_id])
        n_answers = len(examples["answers"][sample_id])
        
        # sample if we get more pairs than maximum
        if len(pairs) > MAX_PAIRS_PER_QUESTION:
            indices = np.random.choice(list(range(len(pairs))), MAX_PAIRS_PER_QUESTION, replace=False)
            pairs = [pairs[i] for i in indices]
        
        # construct the samples
        for pair in pairs:
            for key in examples:
                if key=="question":
                    new_examples[key].append(html2md(examples[key][sample_id]))
                else:
                    new_examples[key].append(examples[key][sample_id])
            new_examples["response_j"].append(html2md(pair[0]))
            new_examples["response_k"].append(html2md(pair[1]))
    return new_examples

In [91]:
ds_result = ds_splits.map(preprocess, batch_size=1000, batched=True, num_proc=60)

Map (num_proc=60):   0%|          | 0/3000000 [00:00<?, ? examples/s]

/opt/conda/envs/jupyter/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Map (num_proc=60):   0%|          | 0/3000000 [00:00<?, ? examples/s]

/opt/conda/envs/jupyter/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Map (num_proc=60):   0%|          | 0/3000000 [00:00<?, ? examples/s]

/opt/conda/envs/jupyter/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/opt/conda/envs/jupyter/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/opt/conda/envs/jupyter/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Map (num_proc=60):   0%|          | 0/1807695 [00:00<?, ? examples/s]

/opt/conda/envs/jupyter/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [92]:
ds_result

DatasetDict({
    finetune: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 7440923
    })
    reward: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 7441998
    })
    rl: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 7435908
    })
    evaluation: Dataset({
        features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 4483004
    })
})

In [93]:
ds_result["finetune"][0]

{'qid': 12891264,
 'question': "I am using jQuery fileupload plugin and I want to do some custom jQuery stuff once fileupload is done\n\nfrom here <https://github.com/blueimp/jQuery-File-Upload/wiki/Options>\n\nNow it says this\n\n```\nCallback for successful upload requests.\n$('#fileupload')\n       .bind('fileuploaddone', function (e, data) {/* ... */})\n\n```\n\nNow I have defined this custom function for testing in my own js file\n\n```\n$('#fileupload').bind('fileuploaddone', function (e, data) {/* ... */\nalert('Hello');\n})\n\n```\n\nBut it's not working.\n\nBut if I edit the main file in here\n\n```\n  // Callback for successful uploads:\n            done: function (e, data) {\n\n```\n\nThen it works.",
 'answers': [{'answer_id': 12891484,
   'author': 'Reflective',
   'author_id': 1686626,
   'author_profile': 'https://Stackoverflow.com/users/1686626',
   'pm_score': 4,
   'selected': True,
   'text': "<p>Looking at the library code, seems all events are renamed removing 'fil

In [94]:
ds_result = ds_result.remove_columns(["answers"])

In [95]:
ds_result

DatasetDict({
    finetune: Dataset({
        features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 7440923
    })
    reward: Dataset({
        features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 7441998
    })
    rl: Dataset({
        features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 7435908
    })
    evaluation: Dataset({
        features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 4483004
    })
})

In [96]:
for key in ds_result:
    print(key)

finetune
reward
rl
evaluation


In [100]:
import os
import time
from multiprocessing import Pool
from tqdm import tqdm

from huggingface_hub import Repository


def save_shard(shard_tuple):
    """Save shard"""
    filename, shard = shard_tuple
    # use to_json instead to save as json file
    shard.to_parquet(filename)


def save_manual_shards(ds, user="lvwerra", remote_dataset_repo="stack-exchange-paired", subfolder="train"):
    """Save sharded data
    Args:
        ds (Dataset): dataset to be saved
        user (str): user name
        remote_dataset_repo (str): remote dataset repository
        out_path (str): path to save the shards"""
    # this will create a folder OUT_PATH that is a clone of REMOTE_DATASET_REPO
    # you can save the shards inside it and do git add/commit/push to push data to the hub
    out_path = remote_dataset_repo
    # if out path doesnt already exist
    if not os.path.exists(out_path):
        repo = Repository(
            local_dir=out_path,
            clone_from=user + "/" + remote_dataset_repo,
            repo_type="dataset",
            private=False,
            use_auth_token=True,
            git_user=user,
        )

    # files will be numerous we save them in a folder called data inside out_path
    if not os.path.exists(out_path):
        os.mkdir(out_path + "/data")
    os.mkdir(out_path + f"/data/{subfolder}")
    
    SHARD_SIZE = 1000 << 20
    if ds._indices is not None:
        dataset_nbytes = ds.data.nbytes * len(ds._indices) / len(ds.data)
    else:
        dataset_nbytes = ds.data.nbytes
    num_shards = int(dataset_nbytes / SHARD_SIZE) + 1
    print(f"Number of shards: {num_shards}")

    print("sharding the dataset")
    t_start = time.time()
    shards = (
        ds.shard(num_shards=num_shards, index=i, contiguous=True)
        for i in range(num_shards)
    )
    # use f"{OUT_PATH}/data/train-{index:05d}-of-{num_shards:05d}.json" instead for json files
    filenames = (
        f"{out_path}/data/{subfolder}/train-{index:05d}-of-{num_shards:05d}.parquet"
        for index in range(num_shards)
    )

    with Pool(16) as p:
        list(
            tqdm(
                p.imap_unordered(save_shard, zip(filenames, shards), chunksize=4),
                total=num_shards,
            )
        )
    print(f"Time to save dataset: {time.time()-t_start:.2f}")
    # to push dataset to hub do: git add/commit/push inside OUT_PATH

In [101]:
for key in ds_result:
    save_manual_shards(ds_result[key], subfolder=key)

Number of shards: 20
sharding the dataset


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:28<00:00,  1.43s/it]


Time to save dataset: 29.15
Number of shards: 20
sharding the dataset


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:22<00:00,  1.15s/it]


Time to save dataset: 23.42
Number of shards: 20
sharding the dataset


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:10<00:00,  1.83it/s]


Time to save dataset: 11.36
Number of shards: 12
sharding the dataset


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:10<00:00,  1.12it/s]


Time to save dataset: 11.13
